In [1]:
# coding: utf-8
import re
import math
import csv
import sys
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from obspy import UTCDateTime, Stream, Trace

%matplotlib inline

In [81]:
def unpackPfile(infile):
    
    with open(infile) as f:
        lines = f.readlines()
    
    tmp = lines[0]
    year = int(tmp[1:5])
    month = int(tmp[5:7])
    day = int(tmp[7:9])
    hour = int(tmp[9:11])
    minute = int(tmp[11:13])
    sec = float(tmp[13:19])
    
    lat_d = float(tmp[19:21])
    lat_m = float(tmp[21:26])
    
    lon_d = float(tmp[26:29])
    lon_m = float(tmp[29:34])
    
    dep = float(tmp[34:40])

    dt = datetime(year,month,day,hour,minute,int(sec//1),int(sec%1 * 1000000))
    mag = float(tmp[40:44])

    pfile_info = {}
    pfile_info["ori_time"] = dt
    pfile_info["mag"] = mag
    pfile_info["lat"] = lat_d + lat_m/60.0
    pfile_info["lon"] = lon_d + lon_m/60.0
    pfile_info["dep"] = dep

    intensity = {}
    arrival_time = {}
    weighting = {}
    pga = {}
    for i in lines[1:]:
        sta = i[:5].strip() # strip 去掉左右空格
        weighting[sta] = int(float(i[35:39]))
        intensity[sta] = int(i[76:77])
        pga[sta] = float(i[78:83])
        arrival_time[sta] = pfile_info["ori_time"].replace(minute=int(i[21:23]),second=0,microsecond=0) + timedelta(seconds=float(i[23:29]))
    pfile_info["intensity"] = intensity
    pfile_info["arrival_time"] = arrival_time
    pfile_info["weighting"] = weighting
    pfile_info["pga"] = pga
    
    return pfile_info

In [82]:
data=unpackPfile("02080054.phn")

ValueError: invalid literal for int() with base 10: ''

In [5]:
data

{'ori_time': datetime.datetime(2018, 2, 8, 0, 54, 9, 720000),
 'mag': 0.0,
 'lat': 24.0645,
 'lon': 121.52,
 'dep': 1.69,
 'intensity': {'TWD': 0,
  'ETM': 0,
  'HWA': 0,
  'ETL': 0,
  'WHF': 0,
  'ESL': 0,
  'EGC': 0,
  'TWT': 0,
  'NNS': 0,
  'ENA': 0,
  'WPL': 0,
  'ENT': 0,
  'EHY': 0,
  'TYC': 0,
  'NTY': 0,
  'WTP': 0},
 'arrival_time': {'TWD': datetime.datetime(2018, 2, 8, 0, 54, 11, 470000),
  'ETM': datetime.datetime(2018, 2, 8, 0, 54, 19, 780000),
  'HWA': datetime.datetime(2018, 2, 8, 0, 54, 19, 320000),
  'ETL': datetime.datetime(2018, 2, 8, 0, 54, 12, 840000),
  'WHF': datetime.datetime(2018, 2, 8, 0, 54, 16, 50000),
  'ESL': datetime.datetime(2018, 2, 8, 0, 54, 21, 540000),
  'EGC': datetime.datetime(2018, 2, 8, 0, 54, 23, 640000),
  'TWT': datetime.datetime(2018, 2, 8, 0, 54, 27, 470000),
  'NNS': datetime.datetime(2018, 2, 8, 0, 54, 27, 860000),
  'ENA': datetime.datetime(2018, 2, 8, 0, 54, 26, 880000),
  'WPL': datetime.datetime(2018, 2, 8, 0, 54, 29, 390000),
  'ENT':

In [7]:
data.keys()

dict_keys(['ori_time', 'mag', 'lat', 'lon', 'dep', 'intensity', 'arrival_time', 'weighting', 'pga'])

In [9]:
data["intensity"].keys()

dict_keys(['TWD', 'ETM', 'HWA', 'ETL', 'WHF', 'ESL', 'EGC', 'TWT', 'NNS', 'ENA', 'WPL', 'ENT', 'EHY', 'TYC', 'NTY', 'WTP'])

In [8]:
# read sta location
with open("c:\\datasrc\\hypo3d.sta","r") as fp:
    fp.readline()
    stalist=fp.readlines()

In [19]:
tmp=stalist[0].strip("\n").split()

In [24]:
stalist

['TAP 25 2.2612130.83    6.0-0.01-0.05 0.00 19010101 21001231 TAP                                                                 \n',
 'HSN02448.0212058.66    0.0 0.06 0.00 0.00 19010101 19910831 HSN0                                                                \n',
 'HSN 2448.0212058.66    0.0 0.06 0.00 0.00 19010101 19910831 HSN                                                                 \n',
 'HSN 2449.69121 0.85   34.0 0.02 0.00 0.00 19910901 21001231 HSN                                                                 \n',
 'TCU024 8.7412041.04   84.0-0.16-0.41 0.00 19010101 19930630 TCU0                                                                \n',
 'TCU 24 8.7412041.04   84.0-0.16-0.41 0.00 19010101 19930630 TCU                                                                 \n',
 'TCU 24 8.7412041.04  -66.0-0.13-0.35 0.00 19930701 21001231 TCU                                                                 \n',
 'CHY02329.7612025.96   27.0-0.11-0.29 0.00 19010101 19

In [13]:
lat_d = float(tmp[4:6])
lat_m = float(tmp[6:11])
lon_d = float(tmp[11:14])
lon_m = float(tmp[14:19])

lat = lat_d + lat_m/60.0
lon = lon_d + lon_m/60.0

In [14]:
print(lat, lon)
tmp

25.037666666666667 121.51383333333334


'TAP 25 2.2612130.83    6.0-0.01-0.05 0.00 19010101 21001231 TAP                                                                 \n'

In [15]:
tmp.split()[-1]

'TAP'

In [25]:
stainfo={}
for i in stalist:
    i=i.strip()
    #print(i)

    lat_d = float(i[4:6])
    lat_m = float(i[6:11])
    lon_d = float(i[11:14])
    lon_m = float(i[14:19])

    lat = lat_d + lat_m/60.0
    lon = lon_d + lon_m/60.0
    sta = i.split()[-1]
    
    stainfo[sta]=[lon , lat]
    

In [28]:
stainfo

{'TAP': [121.51383333333334, 25.037666666666667],
 'HSN0': [120.97766666666666, 24.800333333333334],
 'HSN': [121.01416666666667, 24.828166666666668],
 'TCU0': [120.684, 24.145666666666667],
 'TCU': [120.684, 24.145666666666667],
 'CHY0': [120.43266666666666, 23.496],
 'CHY': [120.43266666666666, 23.496],
 'ALS': [120.81333333333333, 23.508333333333333],
 'TAI': [120.205, 22.993166666666667],
 'PNG': [119.5635, 23.565],
 'KAU0': [120.31583333333333, 22.566166666666668],
 'KAU': [120.31583333333333, 22.566166666666668],
 'HEN0': [120.746, 22.003666666666668],
 'HEN': [120.746, 22.003666666666668],
 'ILA': [121.75616666666667, 24.7635],
 'HWA0': [121.6135, 23.975166666666667],
 'HWA': [121.6135, 23.975166666666667],
 'CHK': [121.37283333333333, 23.097666666666665],
 'TTN': [121.15466666666667, 22.752166666666668],
 'TAW': [120.9035, 22.355833333333333],
 'LAY': [121.55816666666666, 22.037333333333333],
 'NCU': [121.1945, 24.967333333333332],
 'YUS': [120.95916666666666, 23.48733333333333

In [48]:
data["intensity"]

{'TWD': 0,
 'ETM': 0,
 'HWA': 0,
 'ETL': 0,
 'WHF': 0,
 'ESL': 0,
 'EGC': 0,
 'TWT': 0,
 'NNS': 0,
 'ENA': 0,
 'WPL': 0,
 'ENT': 0,
 'EHY': 0,
 'TYC': 0,
 'NTY': 0,
 'WTP': 0}

In [42]:
with open("station.txt","w") as fp:
    for i in data["intensity"].keys():
        aa=i+"\n"
        fp.write(aa)

In [55]:
with open("lonlat.txt","w") as fp:
    for i in data["intensity"].keys():
        aa=str(stainfo[i]).strip("[]")+"\n"
        fp.write(aa)

In [76]:
with open("0609plot.bat","w") as fp:
    GMT=f"psbasemap -R119/123/21/26 -JM17.5 -Ba1f0.5 -P -K -V > 0609-U10710019.ps \n"
    fp.write(GMT)
    GMT=f"grdimage low_res_topo.grd -I+a2 -Ctopo2.cpt -JM -R  -K -O -V >> 0609-U10710019.ps \n"
    fp.write(GMT)        
    GMT=f"psxy TaiwanCounty.gmt -JM -R -W0.5  -K -O -V >> 0609-U10710019.ps \n"
    fp.write(GMT)
    GMT=f"psxy lonlat.txt -JM -R -St0.5 -W1 -G255/255/0  -K -O -V   >> 0609-U10710019.ps \n"
    fp.write(GMT)
    GMT=f"psconvert 0609-U10710019.ps \n"
    fp.write(GMT)
    GMT=f"0609-U10710019.jpg"
    fp.write(GMT)

In [77]:
os.system("0609plot.bat")

1